In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20

Using TensorFlow backend.


In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=8)

In [0]:
input = Input(shape=(img_height, img_width, channel,))

# Block 1
layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
layer1 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer1)
layer1 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer1)
layer1 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer1)
layer1 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_1')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)



# Block 2
layer2 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(layer1)
layer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_7', use_bias=False)(layer2)
layer2 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(layer2)
layer2 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(layer2)
layer2 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_10', use_bias=False)(layer2)
layer2 = BatchNormalization(name='norm_2')(layer2)
layer2 = LeakyReLU(alpha=0.1)(layer2)
layer2 = MaxPooling2D(pool_size=(2, 2))(layer2)

# Block 3
layer3 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(layer2)
layer3 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_12', use_bias=False)(layer3)
layer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(layer3)
layer3 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(layer3)
layer3 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_15', use_bias=False)(layer3)
layer3 = BatchNormalization(name='norm_3')(layer3)
layer3 = LeakyReLU(alpha=0.1)(layer3)
layer3 = MaxPooling2D(pool_size=(2, 2))(layer3)

# Block 4
layer4 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(layer3)
layer4 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_17', use_bias=False)(layer4)
layer4 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(layer4)
layer4 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(layer4)
layer4 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(layer4)
layer4 = BatchNormalization(name='norm_4')(layer4)
layer4 = LeakyReLU(alpha=0.1)(layer4)
layer4 = MaxPooling2D(pool_size=(2, 2))(layer4)

layer1 = Conv2D(32, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_5')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)
layer1 = Lambda(space_to_depth_x2)(layer1)

layers = concatenate([layer1, layer4])
layers = Conv2D(num_classes,2)(layers)
layers = Flatten()(layers)

output = Activation('softmax')(layers)
#output = Conv2D(num_classes, 1, activation='softmax')(layers)



Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 32, 32, 64)   18432       conv_1[0][0]                     
__________________________________________________________________________________________________
conv_3 (Conv2D)                 (None, 32, 32, 128)  73728       conv_2[0][0]                     
__________________________________________________________________________________________________
conv_4 (Co

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 214s 4ms/step - loss: 1.6202 - acc: 0.4553 - val_loss: 1.5959 - val_acc: 0.5034
Epoch 2/50
50000/50000 [==============================] - 199s 4ms/step - loss: 1.0693 - acc: 0.6333 - val_loss: 1.0395 - val_acc: 0.6409
Epoch 3/50
50000/50000 [==============================] - 199s 4ms/step - loss: 0.8867 - acc: 0.6971 - val_loss: 1.0822 - val_acc: 0.6444
Epoch 4/50
50000/50000 [==============================] - 199s 4ms/step - loss: 0.7854 - acc: 0.7310 - val_loss: 1.0059 - val_acc: 0.6647
Epoch 5/50
50000/50000 [==============================] - 199s 4ms/step - loss: 0.7048 - acc: 0.7574 - val_loss: 0.9924 - val_acc: 0.6739
Epoch 6/50
50000/50000 [==============================] - 199s 4ms/step - loss: 0.6423 - acc: 0.7780 - val_loss: 0.9704 - val_acc: 0.6769
Epoch 7/50
50000/50000 [==============================] - 199s 4ms/step - 

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 15s 1ms/step
Test loss: 2.3208101592063906
Test accuracy: 0.679
Saved the model to disk


In [0]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model.h5')

Saved the model to disk


FileNotFoundError: ignored